# Cleanng CDDB

In [1]:
import logging

import pandas as pd
import pandera as pa

import clean_cddb
from clean_cddb.utils import get_failure_cases_summary_as_formatted_table, get_check_func_descriptions

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(process)d - %(levelname)s - %(message)s",
)
filepath = "../data/input/cddb.tsv"
source_df = pd.read_csv(filepath, sep="\t", dtype="str")

## Apply validation checks (pandera schema) and review failure cases

In [2]:
try:
    validated_df = clean_cddb.schema(source_df, lazy=True)
    logging.info("Validation success. No failure cases detected.")
except pa.errors.SchemaErrors as err:
    logging.info("Validation failure. Failure cases detected.")
    logging.debug(err)
    failure_cases_df = err.failure_cases

failure_cases_df = failure_cases_df.pipe(get_check_func_descriptions, clean_cddb.schema)

2023-07-29 22:51:34,228 - 70128 - INFO - Validation failure. Failure cases detected.


`failure_cases_df`
* The `failure_cases_df` shows the name of the column, the check, failure case (example), and row index position of the failure case in the original data frame. 
* The index can support bulk operations such as joining and querying the original dataframe for failure cases or rejecting rows in the set of failure case indices.

In [3]:
(
    failure_cases_df.head().loc[
        :, ["schema_context", "column", "check", "failure_case", "index"]
    ]
)

,schema_context,column,check,failure_case,index
0,Column,artist,not_nullable,NaN,9030
4653,Column,genre,not_nullable,NaN,8088
4664,Column,genre,not_nullable,NaN,8135
4663,Column,genre,not_nullable,NaN,8134
4662,Column,genre,not_nullable,NaN,8127


### Summary of failure cases

Here we see aggregated counts of the number of failure cases for each validation check.

In [4]:
failure_cases_summary = (
    failure_cases_df.groupby(["column", "check"], as_index=False)
    .size()
    .sort_values(by=["column", "check"])
    .rename(columns={"size": "counts"})
)

failure_cases_summary

,column,check,counts
0,artist,Check for *possibly* invalid symbols.,1040
1,artist,Check for invalid artist values.,697
2,artist,not_nullable,1
3,category,Check for invalid categories.,89
4,genre,Check for invalid genres.,1
5,genre,not_nullable,3388
6,id,Check that the length of 'id' is 6 characters.,477
7,title,Check for *possibly* invalid symbols.,1200
8,title,not_nullable,8
9,year,Check if year is numeric.,28


We also have a helper utility function to display the source code along side each check function name.

In [5]:
# Report summary counts
failure_cases_summary_table = get_failure_cases_summary_as_formatted_table(failure_cases_df)

print(failure_cases_summary_table)

+----------+------------------------------------------------+----------+--------------------------------------------------------------------------------------+
| column   | check                                          |   counts | check_source_code                                                                    |
+==========+================================================+==========+======================================================================================+
| artist   | Check for *possibly* invalid symbols.          |     1040 | def check_col_has_valid_characters(x: Any) -> bool:                                  |
|          |                                                |          |     """Check for *possibly* invalid symbols."""                                      |
|          |                                                |          |                                                                                      |
|          |                            

# Cleaning step

We can use the same checks from the pandera validation schema to trigger cleaning actions such as:
* do nothing / ignore the value
* transform the value; e.g., replace value with a substitute (e.g., 'N/A')
* or reject the entire record

Here we apply several cleaning functions on the source_df via .pipe(Callable).
* Each function takes a dataframe and returns a dataframe, so we can chain together the cleaning operations like so.
* Later, we will 
  1. compare `source_df` and `clean_df` as a before/after check
  2. re-apply our validation checks (pandera schema) to the new `clean_df` to verify that our transformations improved our data quality

In [6]:
def df_to_var(df, var_name):
    globals()[var_name] = df
    return df

clean_df = (
source_df
    .pipe(clean_cddb.clean_df_standardize_various_artists)
    .pipe(clean_cddb.clean_df_try_to_fix_encoding_errors, "artist")
    
    .pipe(df_to_var, 'clean_df_artist_transforms_only')
    
    .pipe(clean_cddb.clean_df_invalid_symbols)
    .pipe(clean_cddb.clean_df_invalid_categories)
    .pipe(clean_cddb.clean_df_id_zero_padding)
    .pipe(clean_cddb.clean_df_genre_invalid)
    .pipe(clean_cddb.clean_df_tracks_invalid_symbols)
    .pipe(clean_cddb.clean_df_year)
    .pipe(clean_cddb.clean_df_title)
    .pipe(clean_cddb.clean_df_genre)
    
    # Save an intermediate dataframe prior to dropping records
    # so we can compare with source_df later
    .pipe(df_to_var, 'clean_df_before_drops')
    
    # Drop rows with "REJECT_ROW*" prefix
    .query("~id.str.contains('REJECT_ROW')")
    
    .drop(columns=['merged_values'])
)

In [7]:
# apply schema to clean_df
try:
    validated_df = clean_cddb.schema(clean_df, lazy=True)
    logging.info("Validation success. No failure cases detected.")
except pa.errors.SchemaErrors as err:
    logging.info("Validation failure. Failure cases detected.")
    logging.debug(err)
    after_cleaning_failure_cases_df = err.failure_cases

after_cleaning_failure_cases_df = after_cleaning_failure_cases_df.pipe(
    get_check_func_descriptions, clean_cddb.schema
)

after_cleaning_failure_cases_summary = (
    after_cleaning_failure_cases_df.groupby(["column", "check"], as_index=False)
    .size()
    .sort_values(by=["column", "check"])
    .rename(columns={"size": "counts"})
)

2023-07-29 22:51:35,047 - 70128 - INFO - Validation failure. Failure cases detected.


# Evaluation

Counts of Failure Cases Before vs After Data Cleaning

In [8]:
(
failure_cases_summary
.merge(after_cleaning_failure_cases_summary, 
       on=['column', 'check'], 
       how='outer', 
       suffixes=['_before_cleaning', '_after_cleaning']
       )
.fillna('')
)

,column,check,counts_before_cleaning,counts_after_cleaning
0,artist,Check for *possibly* invalid symbols.,1040,
1,artist,Check for invalid artist values.,697,
2,artist,not_nullable,1,
3,category,Check for invalid categories.,89,
4,genre,Check for invalid genres.,1,
5,genre,not_nullable,3388,
6,id,Check that the length of 'id' is 6 characters.,477,
7,title,Check for *possibly* invalid symbols.,1200,641.0
8,title,not_nullable,8,
9,year,Check if year is numeric.,28,


Comparing `source_df` and `clean_df`

* We will actually use an intermediate dataframe `clean_df_before_drops` that has the same dimensions as our original dataframe.
    * Prior to dropping dirty records, `clean_df_before_drops` has values over-written with a prefix "REJECT_RECORD".
    * This enables easier side by side comparison.
* The final output `clean_df` will actually omit records that we intend to drop.


In [9]:
source_df.head()

,artist,category,genre,title,tracks,year,id,merged_values
0,Backstreet Boys,blues,Pop,Millennium,Larger Than Life | I Want It That Way | Show Me The Meaning Of Being Lonely | It's Gotta Be You | I Need You Tonight | Don't Want You Back | Don't Wanna Lose You Now | The One | Back To Your Heart | Spanish Eyes | No One Else Comes Close | The Perfect Fan | I'll Be There For You | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,NaN,10000,NaN
1,Various,data,NaN,Frankfurt Trance Vol. 04 cd1,DJ Tom Stevens VS. Fridge - Outface 2000 (Radio Mix) | Alice Deejay - Better Off Alone (Signum Remix) | Tillmann Uhrmacher Feat. Peter Ries - Bassfly (Original Mix) | DJ 2 L 8 - Too Late | Time Square - Invisible Girl (Future Breeze Remix) | Cirillo - Across The Soundline | DJ Leon & Jam X - Hold It | Sean Dexter - Synthetica (Extended Mix) | DJ BjÃ¶rn - On A Mission (Original Mix) | 8Voice - Music Hypnotizes 2000 | Alex Apollo - Jahr 2000 | Headroom - Utopia (Radio Mix) | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,NaN,100001,NaN
2,NO RETURN,data,Data,Self Mutilation,Do or Die | Truth and Reality | Lost | Soul Extractor | Sadistic Desire | The True Way | Fanatic Mind | Individualistic Ideal | One Life | Trail of Blood | Sect | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,NaN,100002,NaN
3,Ã¤Â¸Â­Ã¦?â€˜Ã©â€ºâ€¦Ã¤Â¿Å,data,Pop,Ã¦Æ’Â³Ã£?â€žÃ¥â€¡ÂºÃ£?Â®Ã£?â€¹Ã£?â€˜Ã£â€šâ€°,Ã§â€ºâ€ Ã¥Â¸Â°Ã£â€šÅ | Ã£?â€žÃ£?Â¤Ã£?â€¹Ã¨Â¡â€”Ã£?Â§Ã¤Â¼Å¡Ã£?Â£Ã£?Å¸Ã£?ÂªÃ£â€šâ€° | Ã©Â¢Â¨Ã£?Â®Ã£?ÂªÃ£?â€žÃ¦â€”Â¥ | Ã£?Å¸Ã£?Â Ã£?Å Ã¥â€°?Ã£?Å’Ã£?â€žÃ£?â€ž | Ã£?ÂµÃ£â€šÅ’Ã£?â€šÃ£?â€ž | Ã£?â€šÃ£â€š?Ã©?â€™Ã¦ËœÂ¥ | Ã¤Â¿ÂºÃ£?Å¸Ã£?Â¡Ã£?Â®Ã¦â€”â€¦ | Ã§â„¢Â½Ã£?â€žÃ¥Â¯Â«Ã§Å“Å¾Ã©Â¤Â¨ | Ã£?â€¢Ã£?â„¢Ã£â€šâ€°Ã£?â€žÃ¦â„¢â€šÃ¤Â»Â£ | Ã¥Â¤Å“Ã¨Â¡Å’Ã¥Ë†â€”Ã¨Â»Å | Ã£?â€šÃ£?ÂªÃ£?Å¸Ã£â€šâ€™Ã¦â€žâ€ºÃ£?â„¢Ã£â€šâ€¹Ã§Â§? | Ã©?â€™Ã¦ËœÂ¥Ã¨Â²Â´Ã¦â€”? | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,1989,100003,NaN
4,Emanuel,data,Data,Felicidade,Felicidade quando o telefone toca | Vem bailar o tic tic | Quero que sejas minha e de mais ninguem | Eu sei que me amas | O melhor que hÃ¡ | Minha vizinha deixa me a bater mal | S. JoÃ£o Ã© foliÃ£o | SÃ³ quero o teu carinho | tudo farei para ter a tua paixÃ£o | serÃ¡s sempre minha | Vem bailar o tic tic verÃ§Ã£o dance | Mix | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,1998,100004,NaN


In [10]:
clean_df_before_drops.head()

,artist,category,genre,title,tracks,year,id,merged_values
0,Backstreet Boys,blues,Pop,Millennium,Larger Than Life | I Want It That Way | Show Me The Meaning Of Being Lonely | It's Gotta Be You | I Need You Tonight | Don't Want You Back | Don't Wanna Lose You Now | The One | Back To Your Heart | Spanish Eyes | No One Else Comes Close | The Perfect Fan | I'll Be There For You | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,<NA>,010000,NaN
1,Various,N/A,N/A,Frankfurt Trance Vol. 04 cd1,DJ Tom Stevens VS. Fridge - Outface 2000 (Radio Mix) | Alice Deejay - Better Off Alone (Signum Remix) | Tillmann Uhrmacher Feat. Peter Ries - Bassfly (Original Mix) | DJ 2 L 8 - Too Late | Time Square - Invisible Girl (Future Breeze Remix) | Cirillo - Across The Soundline | DJ Leon & Jam X - Hold It | Sean Dexter - Synthetica (Extended Mix) | DJ BjÃ¶rn - On A Mission (Original Mix) | 8Voice - Music Hypnotizes 2000 | Alex Apollo - Jahr 2000 | Headroom - Utopia (Radio Mix) | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,<NA>,100001,NaN
2,NO RETURN,N/A,N/A,Self Mutilation,Do or Die | Truth and Reality | Lost | Soul Extractor | Sadistic Desire | The True Way | Fanatic Mind | Individualistic Ideal | One Life | Trail of Blood | Sect | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,<NA>,100002,NaN
3,REJECT_ROW - invalid artist,N/A,REJECT_ROW - invalid artist,REJECT_ROW - invalid artist,REJECT_ROW - invalid artist,<NA>,REJECT_ROW - invalid artist,REJECT_ROW - invalid artist
4,Emanuel,N/A,N/A,Felicidade,Felicidade quando o telefone toca | Vem bailar o tic tic | Quero que sejas minha e de mais ninguem | Eu sei que me amas | O melhor que hÃ¡ | Minha vizinha deixa me a bater mal | S. JoÃ£o Ã© foliÃ£o | SÃ³ quero o teu carinho | tudo farei para ter a tua paixÃ£o | serÃ¡s sempre minha | Vem bailar o tic tic verÃ§Ã£o dance | Mix | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,1998,100004,NaN


In [11]:
pd.set_option('display.max_colwidth', 50)

comps_df = (
source_df.sort_index()
 .compare(clean_df_before_drops.sort_index(), result_names=('before_cleaning', 'after_cleaning'))
 .astype('object')
 .fillna('')
 )

# Show before/after comps

In [12]:
pd.set_option('display.max_colwidth', None)

columns_to_compare = ['artist', 'category', 'genre', 'title', 'tracks', 'year', 'id']

comps_df_formatted = (
comps_df
 .astype(str) 
 .stack()
 .reset_index()
 .rename(columns={'level_0': 'row_id', 'level_1': 'before_or_after'})
 .drop(columns=['merged_values'])
 .groupby(['row_id'], as_index=False)
    [columns_to_compare]
    .agg(lambda row: '  =>  '.join(row))
 .replace('^(  =>  )$', '', regex=True)
)

comps_df_formatted.sample(10, random_state=12)

,row_id,artist,category,genre,title,tracks,year,id
3211,3607,,,=> N/A,,,,
1465,1647,Various Artists => Various,,,,,,
6838,7657,,,=> N/A,,,,
6138,6868,,,=> N/A,,,,
428,479,,,=> N/A,,,,
7526,8421,,,=> N/A,,,,
5264,5894,,,,,,2003 => 2003,
2197,2463,,,=> N/A,,,,
4396,4913,,,,,,1994 => 1994,
4298,4802,TomÃ¡s San Miguel con Jorge Pardo => Tomás San Miguel con Jorge Pardo,,,,,1993 => 1993,


### Sample transformations

* Here we can see that we transform "Various Artists" and "<various>" to "Various". 
* We also fixed invalid characters converting text from "JÃ¶rg Hilbert & Felix Janosa" to "Jörg Hilbert & Felix Janosa".
* Later, we will do a more comprehensive before/after analysis after applying all the cleaning transformations

In [13]:
example_idxs = [7629, 1822, 117, 4129]
source_df.compare(clean_df_artist_transforms_only).loc[example_idxs, :].fillna(pd.NA)

artist                             
                              self                        other
7629               Various Artists                      Various
1822  JÃ¶rg Hilbert & Felix Janosa  Jörg Hilbert & Felix Janosa
117                Various Artists                      Various
4129                     <various>                      Various

In [14]:
comps_df.sample(10, random_state=3)

artist                       category                 \
     before_cleaning after_cleaning before_cleaning after_cleaning   
3321                                                                 
2710                                                                 
9416                                                                 
5715                                                                 
3713                                                                 
6795                                                                 
1885                                                                 
4251                                                                 
6576                                                                 
6065                                                                 

               genre                          title                 \
     before_cleaning after_cleaning before_cleaning after_cleaning   
3321                            N/A                                  
2710                            N/A                                  
9416                                                                 
5715                                                                 
3713                                                                 
6795                            N/A                                  
1885                            N/A                                  
4251                                                                 
6576                                                                 
6065                                                                 

              tracks                           year                 \
     before_cleaning after_cleaning before_cleaning after_cleaning   
3321                                           2003           2003   
2710                                                                 
9416                                           1979           1979   
5715                                           2000           2000   
3713                                           1999           1999   
6795                                                                 
1885                                                                 
4251                                           1996           1996   
6576                                           1997           1997   
6065                                           2001           2001   

                  id                  merged_values                 
     before_cleaning after_cleaning before_cleaning after_cleaning  
3321                                                                
2710                                                                
9416            4348         004348                                 
5715                                                                
3713                                                                
6795                                                                
1885                                                                
4251                                                                
6576                                                                
6065

In [15]:
comps_df.sample(5, random_state=4)

artist                                     category  \
               before_cleaning               after_cleaning before_cleaning   
5208                                                                          
6279                                                                          
4088                                                                          
3886  C Lanzbom & Noah Solomon  REJECT_ROW - invalid artist            misc   
6165               MÃ¼llerbeat                   Müllerbeat                   

                              genre                               \
     after_cleaning before_cleaning               after_cleaning   
5208                                                               
6279                                                               
4088                                                               
3886            N/A            Misc  REJECT_ROW - invalid artist   
6165                                                         N/A   

               title                               \
     before_cleaning               after_cleaning   
5208                                                
6279                                                
4088                                                
3886       Butterfly  REJECT_ROW - invalid artist   
6165                                                

                                                                                                                                                                                                                                                                                                                                                                                                                                                             tracks  \
                                                                                                                                                                                                                                                                                                                                                                                                                                                    before_cleaning   
5208                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
6279                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
4088                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
3886  Ani L'Dodi | Maggie in Jerusalem | Shiru Lo | Esh Kodesh | Shomer Yisrael/Irish Melodie | Butterfly | Wild & Free | Sameach | Ein Ketokenu | Freize Britches | Sabenu | Havdalah | Madhouse |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 

Smaller before/after example
* Here we can see that we transform "Various Artists" and "<various>" to "Various". 
* We also fixed invalid characters converting text from "JÃ¶rg Hilbert & Felix Janosa" to "Jörg Hilbert & Felix Janosa".
* Later, we will do a more comprehensive before/after analysis after applying all the cleaning transformations

In [16]:
example_idxs = [7629, 1822, 117, 4129]
source_df.compare(clean_df_artist_transforms_only).loc[example_idxs, :].fillna(pd.NA)

artist                             
                              self                        other
7629               Various Artists                      Various
1822  JÃ¶rg Hilbert & Felix Janosa  Jörg Hilbert & Felix Janosa
117                Various Artists                      Various
4129                     <various>                      Various

### Sample transformations

* Here we can see that we transform "Various Artists" and "<various>" to "Various". 
* We also fixed invalid characters converting text from "JÃ¶rg Hilbert & Felix Janosa" to "Jörg Hilbert & Felix Janosa".
* Later, we will do a more comprehensive before/after analysis after applying all the cleaning transformations

In [17]:
example_idxs = [7629, 1822, 117, 4129]
source_df.compare(clean_df_artist_transforms_only).loc[example_idxs, :].fillna(pd.NA)

artist                             
                              self                        other
7629               Various Artists                      Various
1822  JÃ¶rg Hilbert & Felix Janosa  Jörg Hilbert & Felix Janosa
117                Various Artists                      Various
4129                     <various>                      Various

# Transform to track-level data

We want a separate track-level dataset that can be joined to the album-level data in `clean_df`.

In [18]:
# Transform to track-level data
track_level_df = (
    # Start with original dataframe
    clean_df
    # Split 'tracks' on pipe into an array; we can "explode" it later
    .pipe(lambda _df: _df.assign(tracks=_df["tracks"].str.split("|")))
    # "explode"/expand from "tracks" array in to 1 observation per track
    # perform a self-join to CD data set; the CD-level data will repeat for each track
    .pipe(
        lambda _df: _df.merge(
            _df["tracks"].explode(), left_index=True, right_index=True
        )
    )
    .pipe(df_to_var, "df_after_explode")
    # Make new 'tracks' field; strip ' ' empty space track names to '' empty string
    .pipe(lambda _df: _df.assign(tracks=_df["tracks_y"].str.strip()))
    # Don't need these fields anymore
    .drop(columns=["tracks_x", "tracks_y"])
    # Filter out empty string track names
    .query("tracks!=''")
    .pipe(df_to_var, "df_after_empty_track_name_filter")
    .reset_index(drop=True)
    .loc[:, ['id', 'tracks']]
    .reset_index()
    .rename(columns={'id': 'album_row_id', 
                    'index': 'track_id',
                    'tracks': 'track_name',
                    }
            )
)

In [20]:
# Demo joining track_level_df and clean_df
(track_level_df
 .head(15)
 .merge(clean_df, left_on=['album_row_id'], right_on=['id'], how='inner')
 .loc[:, ['album_row_id', 'track_id', 'title', 'track_name']]
)

,album_row_id,track_id,title,track_name
0,010000,0,Millennium,Larger Than Life
1,010000,1,Millennium,I Want It That Way
2,010000,2,Millennium,Show Me The Meaning Of Being Lonely
3,010000,3,Millennium,It's Gotta Be You
4,010000,4,Millennium,I Need You Tonight
5,010000,5,Millennium,Don't Want You Back
6,010000,6,Millennium,Don't Wanna Lose You Now
7,010000,7,Millennium,The One
8,010000,8,Millennium,Back To Your Heart
9,010000,9,Millennium,Spanish Eyes


# End